# SDWFS–SSDF AGN $dN/dm$ Curves
Author: Benjamin Floyd

Compares the number count distribution curves for both the SDWFS and SSDF IRAC fields

In [1]:
import json
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.table import Table, setdiff
from astropy.io import fits
from astropy.wcs import WCS
from scipy.interpolate import interp1d
from astropy.coordinates import SkyCoord
from matplotlib.colors import TABLEAU_COLORS
import matplotlib as mpl

In [2]:
# Set our selection magnitude ranges
ch1_bright_mag = 10.0  # Bright-end 3.6 um magnitude
ch1_faint_mag = 18.3  # Faint-end 3.6 um magnitude
ch2_bright_mag = 10.45  # Bright-end 4.5 um magnitude
ch2_faint_mag = 17.48  # Faint-end 4.5 um magnitude

# Set our magnitude binning
mag_bin_width = 0.25
magnitude_bins = np.arange(10., 18., mag_bin_width)
magnitude_bin_centers = magnitude_bins[:-1] + np.diff(magnitude_bins) / 2

In [3]:
# Read in the color threshold--redshift relations
with open('Data_Repository/Project_Data/SPT-IRAGN/SDWFS_background/SDWFS_purity_color_4.5_17.48.json', 'r') as f:
    sdwfs_purity_data = json.load(f)
z_bins = sdwfs_purity_data['redshift_bins'][:-1]
agn_purity_color = interp1d(z_bins, sdwfs_purity_data['purity_90_colors'], kind='previous')

selection_colors = np.array([agn_purity_color(z) for z in z_bins])

In [4]:
# Read in the SDWFS IRAGN catalog
sdwfs_agn = Table.read('Data_Repository/Project_Data/SPT-IRAGN/Output/SDWFS_full-field_IRAGN.fits')

# Read in the SSDF IRAC catalog
ssdf_template = Table.read('Data_Repository/Project_Data/SPT-IRAGN/SPTPol/catalogs/ssdf_table_template.cat', format='ascii.sextractor')
ssdf_gals = Table.read('Data_Repository/Catalogs/SSDF/SSDF2.20170125.v10.public.cat', format='ascii', names=ssdf_template.colnames)

In [5]:
# Read in the Gaia catalogs for both fields
sdwfs_gaia = Table.read('Data_Repository/Catalogs/Bootes/SDWFS/SDWFS_Gaia.fits')
ssdf_gaia = Table.read('Data_Repository/Catalogs/SSDF/SSDF_Gaia_sources.fits')

sdwfs_gaia = sdwfs_gaia[~(sdwfs_gaia['in_qso_candidates'] | sdwfs_gaia['in_galaxy_candidates'])]
ssdf_gaia = ssdf_gaia[~(ssdf_gaia['in_qso_candidates'] | ssdf_gaia['in_galaxy_candidates'])]

In [6]:
# Match the Gaia sources to the catalogs
sdwfs_agn_coords = SkyCoord(sdwfs_agn['ALPHA_J2000'], sdwfs_agn['DELTA_J2000'], unit=u.deg)
sdwfs_gaia_coords = SkyCoord(sdwfs_gaia['ra'], sdwfs_gaia['dec'], unit=u.deg)

ssdf_gal_coords = SkyCoord(ssdf_gals['ALPHA_J2000'], ssdf_gals['DELTA_J2000'], unit=u.deg)
ssdf_gaia_coords = SkyCoord(ssdf_gaia['ra'], ssdf_gaia['dec'], unit=u.deg)

sdwfs_idx, sdwfs_sep, _ = sdwfs_gaia_coords.match_to_catalog_sky(sdwfs_agn_coords)
ssdf_idx, ssdf_sep, _ = ssdf_gaia_coords.match_to_catalog_sky(ssdf_gal_coords)

sdwfs_agn = setdiff(sdwfs_agn, sdwfs_agn[sdwfs_idx[sdwfs_sep <= 1. * u.arcsec]])
ssdf_gals = setdiff(ssdf_gals, ssdf_gals[ssdf_idx[ssdf_sep <= 1 * u.arcsec]])

In [7]:
# We need to select the AGN in the SSDF catalog
ssdf_agn = ssdf_gals[(ch1_bright_mag < ssdf_gals['I1_MAG_APER4']) & (ssdf_gals['I1_MAG_APER4'] <= ch1_faint_mag) &
                      (ch2_bright_mag < ssdf_gals['I2_MAG_APER4']) & (ssdf_gals['I2_MAG_APER4'] <= ch2_faint_mag)]

# Instead of running the selection membership values for all 9 thresholds we will simply take a color cut and create columns with flags designating
# if the galaxy would be an agn or not using that threshold
for selection_color in selection_colors:
    ssdf_agn[f'is_agn_{selection_color:.2f}'] = ssdf_agn['I1_MAG_APER4'] - ssdf_agn['I2_MAG_APER4'] >= selection_color

In [8]:
# Read in the SDWFS mask image and WCS
sdwfs_mask_img, sdwfs_mask_hdr = fits.getdata('Data_Repository/Project_Data/SPT-IRAGN/Masks/SDWFS/'
                                              'SDWFS_full-field_cov_mask11_11.fits', header=True)
sdwfs_wcs = WCS(sdwfs_mask_hdr)

# Determine the area of the mask
sdwfs_area = np.count_nonzero(sdwfs_mask_img) * sdwfs_wcs.proj_plane_pixel_area()

# For SSDF, we will just use the approximate area as reported in Ashby+13
ssdf_area = 94 * u.deg**2

# Set the weights
sdwfs_weight = sdwfs_area.value * mag_bin_width
ssdf_weight = ssdf_area.value * mag_bin_width

In [9]:
# Cycle through the color selections to build the dN/dm distributions
dndm_dists = {}
for selection_color in selection_colors:
    # Isolate the AGN for the selection threshold
    sdwfs_agn_z = sdwfs_agn[sdwfs_agn[f'SELECTION_MEMBERSHIP_{selection_color:.2f}'] >= 0.5]
    ssdf_agn_z = ssdf_agn[ssdf_agn[f'is_agn_{selection_color:.2f}']]

    # Compute the number count distributions
    sdwfs_dndm, _ = np.histogram(sdwfs_agn_z['I2_MAG_APER4'], bins=magnitude_bins, weights=sdwfs_agn_z['COMPLETENESS_CORRECTION'])
    sdwfs_dndm_weighted = sdwfs_dndm / sdwfs_weight

    ssdf_dndm, _ = np.histogram(ssdf_agn_z['I2_MAG_APER4'], bins=magnitude_bins)
    ssdf_dndm_weighted = ssdf_dndm / ssdf_weight

    # Compute the associated errors
    sdwfs_dndm_err = np.sqrt(sdwfs_dndm) / sdwfs_weight
    ssdf_dndm_err = np.sqrt(ssdf_dndm) / ssdf_weight

    dndm_dists[f'color_{selection_color:.2f}'] = {'sdwfs_dndm': sdwfs_dndm_weighted, 'sdwfs_dndm_err': sdwfs_dndm_err,
                                                  'ssdf_dndm': ssdf_dndm_weighted, 'ssdf_dndm_err': ssdf_dndm_err}

In [13]:
with mpl.rc_context({'font.size': 20}):
    fig, axarr = plt.subplots(nrows=5, ncols=2, sharex='col', sharey='row', figsize=(20, 50), tight_layout=True)
    for ax, selection_color, plot_color in zip(axarr.flatten(), selection_colors, list(TABLEAU_COLORS.keys())):
        dndm_data = dndm_dists[f'color_{selection_color:.2f}']
        ax.errorbar(magnitude_bin_centers, dndm_data['sdwfs_dndm'], yerr=dndm_data['sdwfs_dndm_err'], xerr=mag_bin_width/2, fmt='o', markersize=10, 
                    color=plot_color, label=fr'SDWFS')
        ax.errorbar(magnitude_bin_centers, dndm_data['ssdf_dndm'], yerr=dndm_data['ssdf_dndm_err'], xerr=mag_bin_width/2, fmt='^', markersize=10, 
                    color=plot_color, label=fr'SSDF')
        ax.set(xlabel='[4.5] (Vega)', ylabel=r'$dN/dm$ [deg$^{-2}$ mag$^{-1}$]', yscale='log', title=fr'$[3.6] - [4.5] \geq {selection_color:.2f}$')
        ax.legend()
        ax.label_outer()
    fig.savefig('Data_Repository/Project_Data/SPT-IRAGN/local_backgrounds/plots/SDWFS/SDWFS-SSDF_dndm_all_color_thresholds.pdf')